# NotebookA: Two-Phase Snipper v8 - MINIMAL PROCESSING

**VERSION: 8.0 | DATE: 2025-11-29**

## Only the non-negotiables:
1. Load raw (NO broadband prefilter)
2. Channel selection (frontal vs posterior)
3. Average reference (makes data interpretable)
4. Gamma extraction (30-100 Hz) → envelope → spikes
5. Jitter/collapse/rise detection

**The gamma bandpass is the ONLY filter.**

NO notch. NO ICA. NO broadband prefilter.

In [6]:
import mne
import numpy as np
import pandas as pd
from scipy.signal import hilbert, find_peaks
from pathlib import Path
import warnings
warnings.filterwarnings('ignore')
mne.set_log_level('ERROR')

print('NotebookA v8 - MINIMAL PROCESSING')
print('No broadband prefilter. Gamma extraction is the only filter.')

NotebookA v8 - MINIMAL PROCESSING
No broadband prefilter. Gamma extraction is the only filter.


In [2]:
DATA_FOLDER = Path(r'C:\Users\bwilk\Downloads\dmt_inhalation_study3992359')
OUTPUT_DIR = DATA_FOLDER / 'gold_v8'
OUTPUT_DIR.mkdir(parents=True, exist_ok=True)

files = list(DATA_FOLDER.glob('*.bdf')) + list(DATA_FOLDER.glob('*.edf')) + list(DATA_FOLDER.glob('*.fif'))
files = [f for f in files if 'clean' not in f.name.lower() and 'gold' not in f.name.lower()]

print(f'Found {len(files)} raw files')
print(f'Output: {OUTPUT_DIR}')

Found 69 raw files
Output: C:\Users\bwilk\Downloads\dmt_inhalation_study3992359\gold_v8


In [3]:
FRONTAL_CANDIDATES = ['Fp1', 'Fp2', 'AF7', 'AF8', 'Fz', 'F3', 'F4', 'F7', 'F8', 'AF3', 'AF4', 'Fpz']
POSTERIOR_CANDIDATES = ['Oz', 'O1', 'O2', 'Pz', 'P3', 'P4', 'PO7', 'PO8', 'P7', 'P8', 'POz', 'PO3', 'PO4']

In [4]:
GAMMA_LOW, GAMMA_HIGH = 30, 100

SPIKE_PERCENTILE = 93
MIN_SPIKE_DISTANCE_SEC = 0.5

PRE_WINDOW_SEC = 30
POST_WINDOW_SEC = 30
WINDOW_DURATION_SEC = 60

# THRESHOLDS
MIN_PRE_JITTER = 0.4
MIN_PRE_SPIKES = 8
MAX_POST_SPIKE_RATIO = 0.5
MIN_POSTERIOR_RISE = 1.2

print(f'Jitter >= {MIN_PRE_JITTER}')
print(f'Collapse <= {MAX_POST_SPIKE_RATIO}')
print(f'Rise >= {MIN_POSTERIOR_RISE}x')

Jitter >= 0.4
Collapse <= 0.5
Rise >= 1.2x


In [5]:
all_metadata = []
all_diagnostics = []

for file_idx, f in enumerate(files):
    print(f'[{file_idx+1}/{len(files)}] {f.name}')
    
    try:
        # STEP 1: LOAD RAW - NO FILTERING
        if f.suffix == '.fif':
            raw = mne.io.read_raw_fif(f, preload=True, verbose=False)
        elif f.suffix == '.bdf':
            raw = mne.io.read_raw_bdf(f, preload=True, verbose=False)
        else:
            raw = mne.io.read_raw_edf(f, preload=True, verbose=False)
        
        sfreq = raw.info['sfreq']
        ch_names = raw.ch_names
        duration = raw.times[-1]
        
        # STEP 2: CHANNEL SELECTION
        frontal_chs = [ch for ch in FRONTAL_CANDIDATES if ch in ch_names]
        if not frontal_chs:
            frontal_chs = [ch for ch in ch_names if 'F' in ch.upper() and 'C' not in ch.upper()][:4]
        if not frontal_chs:
            continue
        
        posterior_chs = [ch for ch in POSTERIOR_CANDIDATES if ch in ch_names]
        if not posterior_chs:
            posterior_chs = [ch for ch in ch_names if ch.startswith('O') or ch.startswith('P')][:4]
        if not posterior_chs:
            continue
        
        # STEP 3: AVERAGE REFERENCE
        eeg_chs = [ch for ch in ch_names if not any(x in ch.lower() for x in ['stim', 'status', 'trigger', 'eog', 'ecg', 'emg', 'gsr', 'resp', 'temp'])]
        if len(eeg_chs) > 0:
            raw.set_eeg_reference('average', ch_type='eeg', verbose=False)
        
        # STEP 4: GAMMA EXTRACTION (the ONLY filter)
        frontal_data = raw.copy().pick(frontal_chs).get_data().mean(axis=0)
        ocular_gamma = mne.filter.filter_data(frontal_data, sfreq, GAMMA_LOW, GAMMA_HIGH, verbose=False)
        ocular_env = np.abs(hilbert(ocular_gamma))
        
        posterior_data = raw.copy().pick(posterior_chs).get_data().mean(axis=0)
        posterior_gamma = mne.filter.filter_data(posterior_data, sfreq, GAMMA_LOW, GAMMA_HIGH, verbose=False)
        posterior_env = np.abs(hilbert(posterior_gamma))
        
        # STEP 5: SPIKE DETECTION
        spike_height = np.percentile(ocular_env, SPIKE_PERCENTILE)
        min_distance = int(MIN_SPIKE_DISTANCE_SEC * sfreq)
        all_peaks, _ = find_peaks(ocular_env, height=spike_height, distance=min_distance)
        
        if len(all_peaks) < MIN_PRE_SPIKES + 3:
            continue
        
        # STEP 6: JITTER / COLLAPSE / RISE
        all_peaks_sec = all_peaks / sfreq
        base = f.stem
        window_count = 0
        
        for i, trigger_peak in enumerate(all_peaks):
            trigger_time = trigger_peak / sfreq
            
            if trigger_time < PRE_WINDOW_SEC + 5 or trigger_time + POST_WINDOW_SEC + 5 > duration:
                continue
            
            pre_start = trigger_time - PRE_WINDOW_SEC
            pre_spikes = all_peaks_sec[(all_peaks_sec >= pre_start) & (all_peaks_sec < trigger_time)]
            
            if len(pre_spikes) < MIN_PRE_SPIKES:
                continue
            
            pre_isis = np.diff(pre_spikes)
            if len(pre_isis) < 3:
                continue
            pre_jitter = np.std(pre_isis) / (np.mean(pre_isis) + 1e-12)
            
            post_end = trigger_time + POST_WINDOW_SEC
            post_spikes = all_peaks_sec[(all_peaks_sec > trigger_time) & (all_peaks_sec <= post_end)]
            
            pre_spike_count = len(pre_spikes)
            post_spike_count = len(post_spikes)
            spike_ratio = post_spike_count / (pre_spike_count + 1e-12)
            
            pre_samples = int(PRE_WINDOW_SEC * sfreq)
            post_samples = int(POST_WINDOW_SEC * sfreq)
            pre_posterior = np.mean(posterior_env[trigger_peak - pre_samples:trigger_peak])
            post_posterior = np.mean(posterior_env[trigger_peak:trigger_peak + post_samples])
            posterior_rise = post_posterior / (pre_posterior + 1e-12)
            
            all_diagnostics.append({
                'file': f.name, 'trigger_time': trigger_time,
                'pre_spikes': pre_spike_count, 'post_spikes': post_spike_count,
                'spike_ratio': spike_ratio, 'pre_jitter': pre_jitter,
                'posterior_rise': posterior_rise
            })
            
            if pre_jitter >= MIN_PRE_JITTER and spike_ratio <= MAX_POST_SPIKE_RATIO and posterior_rise >= MIN_POSTERIOR_RISE:
                window_start = trigger_time
                window_end = trigger_time + WINDOW_DURATION_SEC
                if window_end > duration:
                    continue
                
                snippet = raw.copy().crop(tmin=window_start, tmax=window_end)
                outname = f'{base}_BREAKTHROUGH_{window_count+1:02d}_raw.fif'
                outpath = OUTPUT_DIR / outname
                snippet.save(outpath, overwrite=True, verbose=False)
                window_count += 1
                
                print(f'  *** BREAKTHROUGH #{window_count} @{trigger_time:.1f}s | j={pre_jitter:.2f} c={spike_ratio:.2f} r={posterior_rise:.2f}x')
                
                all_metadata.append({
                    'source_file': f.name, 'gold_file': outname,
                    'trigger_time_sec': trigger_time,
                    'pre_spike_count': pre_spike_count, 'post_spike_count': post_spike_count,
                    'spike_collapse_ratio': spike_ratio, 'pre_jitter_cov': pre_jitter,
                    'posterior_rise': posterior_rise,
                    'frontal_channels': ','.join(frontal_chs),
                    'posterior_channels': ','.join(posterior_chs)
                })
    except Exception as e:
        print(f'  ERROR: {e}')

print(f'\nDone. {len(all_diagnostics)} candidates analyzed.')

[1/69] S01-DMT.bdf
[2/69] S01-EC.bdf
[3/69] S02_DMT.bdf
[4/69] S02_EC.bdf
[5/69] S03-DMT.bdf
[6/69] S03-EC.bdf
[7/69] s04-DMT.bdf
[8/69] s04-EC.bdf
[9/69] s05-DMT.bdf
[10/69] s05-EC.bdf
[11/69] S06_DMT.bdf
[12/69] S06_EC.bdf
[13/69] S07_DMT.bdf
[14/69] S07_EC.bdf
[15/69] S08_DMT.bdf
[16/69] S08_EC.bdf
[17/69] S09_DMT.bdf
  *** BREAKTHROUGH #1 @37.8s | j=0.45 c=0.44 r=1.30x
  *** BREAKTHROUGH #2 @39.2s | j=0.48 c=0.44 r=1.32x
  *** BREAKTHROUGH #3 @40.2s | j=0.47 c=0.42 r=1.34x
  *** BREAKTHROUGH #4 @42.9s | j=0.46 c=0.50 r=1.45x
  *** BREAKTHROUGH #5 @48.9s | j=0.54 c=0.50 r=1.64x
  *** BREAKTHROUGH #6 @59.4s | j=0.79 c=0.50 r=2.02x
  *** BREAKTHROUGH #7 @62.9s | j=0.75 c=0.50 r=2.47x
  *** BREAKTHROUGH #8 @102.9s | j=0.84 c=0.45 r=1.24x
  *** BREAKTHROUGH #9 @103.8s | j=0.86 c=0.36 r=1.29x
  *** BREAKTHROUGH #10 @104.3s | j=0.90 c=0.25 r=1.29x
  *** BREAKTHROUGH #11 @107.8s | j=0.99 c=0.17 r=1.44x
  *** BREAKTHROUGH #12 @110.8s | j=0.91 c=0.08 r=1.59x
  *** BREAKTHROUGH #13 @113.0s | 

In [ ]:
if all_diagnostics:
    diag_df = pd.DataFrame(all_diagnostics)
    print(f'Total candidates: {len(diag_df)}')
    print(f'\nJitter - 95th: {diag_df["pre_jitter"].quantile(0.95):.3f}, Max: {diag_df["pre_jitter"].max():.3f}')
    print(f'Collapse - 5th: {diag_df["spike_ratio"].quantile(0.05):.3f}, Min: {diag_df["spike_ratio"].min():.3f}')
    print(f'Rise - 95th: {diag_df["posterior_rise"].quantile(0.95):.3f}, Max: {diag_df["posterior_rise"].max():.3f}')
    
    diag_df.to_csv(OUTPUT_DIR / 'diagnostic_all_candidates.csv', index=False)
    
    diag_df['score'] = diag_df['pre_jitter'] * diag_df['posterior_rise'] / (diag_df['spike_ratio'] + 0.1)
    print(f'\nTOP 20:')
    for i, row in diag_df.nlargest(20, 'score').iterrows():
        print(f"  {row['file'][:20]:<20} @{row['trigger_time']:>5.1f}s | j={row['pre_jitter']:.2f} c={row['spike_ratio']:.2f} r={row['posterior_rise']:.2f}")

In [ ]:
print('='*70)
print('BREAKTHROUGH WINDOWS')
print('='*70)

if all_metadata:
    meta_df = pd.DataFrame(all_metadata)
    meta_df.to_csv(OUTPUT_DIR / 'breakthrough_metadata.csv', index=False)
    print(f'Total: {len(meta_df)}')
    for i, row in meta_df.iterrows():
        print(f"  {row['gold_file']} | j={row['pre_jitter_cov']:.2f} c={row['spike_collapse_ratio']:.2f} r={row['posterior_rise']:.2f}x")
else:
    print('No breakthroughs found')

print(f'\nOutput: {OUTPUT_DIR}')